# Planner quality

This notebook compares planning strategies on small circuits (≤ 20 qubits).

In [ ]:
import time, random, importlib, subprocess, sys, numpy as np
from quasar.cost import CostEstimator, Cost
from quasar.planner import Planner, _supported_backends, _simulation_cost, _add_cost, _better, PlanStep
from quasar.circuit import Circuit
from benchmarks import circuits as circuit_lib

# ensure required libraries
for pkg in ["pandas", "matplotlib"]:
    if importlib.util.find_spec(pkg) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])
import pandas as pd
import matplotlib.pyplot as plt


def build_boundaries(gates):
    n=len(gates)
    prefix=[set()]
    running=set()
    for g in gates:
        running |= set(g.qubits)
        prefix.append(running.copy())
    future=[set() for _ in range(n+1)]
    running=set()
    for i in range(n-1,-1,-1):
        running |= set(gates[i].qubits)
        future[i]=running.copy()
    boundaries=[prefix[i] & future[i] for i in range(n+1)]
    return boundaries


def exhaustive_plan(gates, estimator):
    n=len(gates)
    boundaries=build_boundaries(gates)
    best_cost=None
    def recurse(i, prev_backend, cost):
        nonlocal best_cost
        if i==n:
            if best_cost is None or _better(cost,best_cost):
                best_cost=cost
            return
        for j in range(i+1,n+1):
            segment=gates[i:j]
            backends=_supported_backends(segment)
            num_qubits=len({q for g in segment for q in g.qubits})
            num_gates=j-i
            for backend in backends:
                sim_cost=_simulation_cost(estimator, backend, num_qubits, num_gates)
                conv_cost=Cost(0.0,0.0)
                if prev_backend is not None and prev_backend!=backend and boundaries[i]:
                    bsize=len(boundaries[i])
                    conv_est=estimator.conversion(prev_backend, backend, num_qubits=bsize, rank=2**bsize, frontier=bsize)
                    conv_cost=conv_est.cost
                total=_add_cost(_add_cost(cost, conv_cost), sim_cost)
                recurse(j, backend, total)
    recurse(0,None,Cost(0.0,0.0))
    return best_cost


def greedy_plan(gates, estimator):
    n=len(gates)
    boundaries=build_boundaries(gates)
    total=Cost(0.0,0.0)
    i=0
    prev_backend=None
    while i<n:
        best_inc=None
        best_end=None
        best_backend=None
        for j in range(i+1,n+1):
            segment=gates[i:j]
            backends=_supported_backends(segment)
            num_qubits=len({q for g in segment for q in g.qubits})
            num_gates=j-i
            for backend in backends:
                sim_cost=_simulation_cost(estimator, backend, num_qubits, num_gates)
                conv_cost=Cost(0.0,0.0)
                if prev_backend is not None and prev_backend!=backend and boundaries[i]:
                    bsize=len(boundaries[i])
                    conv_est=estimator.conversion(prev_backend, backend, num_qubits=bsize, rank=2**bsize, frontier=bsize)
                    conv_cost=conv_est.cost
                inc=_add_cost(conv_cost, sim_cost)
                if best_inc is None or _better(inc, best_inc):
                    best_inc=inc
                    best_end=j
                    best_backend=backend
        total=_add_cost(total, best_inc)
        prev_backend=best_backend
        i=best_end
    return total


In [ ]:
est=CostEstimator()
planner=Planner(estimator=est, top_k=2, quick_max_qubits=None, quick_max_gates=None, quick_max_depth=None)

results=[]
for seed in range(10):
    num_qubits = random.randint(2,20)
    circuit = circuit_lib.random_circuit(num_qubits, seed=seed)
    gates=circuit.gates[:8]
    c=Circuit(gates)
    start=time.perf_counter(); oracle=exhaustive_plan(gates, est); t_oracle=time.perf_counter()-start
    start=time.perf_counter(); planres=planner.plan(c, use_cache=False); t_dp=time.perf_counter()-start
    dp_cost=planres.table[-1][planres.final_backend].cost if planres.table else Cost(0.0,0.0)
    start=time.perf_counter(); greedy=greedy_plan(gates, est); t_greedy=time.perf_counter()-start
    results.append({
        "seed":seed,
        "qubits":c.num_qubits,
        "gates":len(gates),
        "oracle":oracle.time,
        "dp":dp_cost.time,
        "greedy":greedy.time,
        "t_oracle":t_oracle,
        "t_dp":t_dp,
        "t_greedy":t_greedy,
        "gap":dp_cost.time/oracle.time - 1,
    })

df=pd.DataFrame(results)
df


In [ ]:
sorted_gaps=np.sort(df["gap"])
ys=np.arange(1,len(sorted_gaps)+1)/len(sorted_gaps)
plt.step(sorted_gaps, ys, where="post")
plt.xlabel("Relative cost gap (DP vs oracle)")
plt.ylabel("CDF")
plt.tight_layout()
